### Load packages etc, and load the child-class of gmspython, "Abatement", and load "techdata_to_tree" which converts a technology catalog into production trees

In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(py['main'])
import abatement, sys, ShockFunction
os.chdir(curr)
data_folder = os.getcwd()+'\\Data'
gams_folder = data_folder + "\\..\\gamsmodels\\Main"

The file_gams_py_gdb0.gdx is still active and was not deleted.
The file_gams_py_gdb1.gdx is still active and was not deleted.
The file_gams_py_gdb3.gdx is still active and was not deleted.


In [2]:
m = abatement.abate(pickle_path=gams_folder+"/gmspython_Abatement", work_folder=work_folder)

In [3]:
m.model.functions = {"std_pdf":"$FUNCTION std_pdf({x}): ((1/(sqrt(2*Pi)))*exp(-(Sqr({x}))/2)) $ENDFUNCTION"}

In [4]:
m.setstate("EOP")

In [5]:
m.reset_settings()

In [6]:
m.write_and_run(name="first")

In [7]:
m.model_instances["first"].__dict__

{'execute_name': 'CollectAndRun.gms',
 'name': 'gmodel',
 'settings': <DB2Gams_l2.gams_settings at 0x28bdf26e448>,
 'export_settings': {'dropattrs': ['settings', 'opt', 'job'],
  'pklattrs': {'settings': 'gams_settings'},
  'opt': 'conopt4.opt'},
 'opt': <gams.options.GamsOptions at 0x28be2cf1148>,
 'opt_file': 'conopt4.opt',
 'import_settings': {},
 'job': <gams.execution.GamsJob at 0x28be2cf1248>,
 'out_db': <DataBase.GPM_database at 0x28be2d03a08>,
 'modelstat': 16.0,
 'solvestat': 1.0}

In [8]:
# m.setstate("ID")
# m.reset_settings()

In [9]:
# # m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
# # m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001

# m.write_and_run(name="ID", kwargs_init={'check_variables':True})
# if m.model_instances["ID"].modelstat == 16.0:
#     print("\nSuccess! The modelstat was 16.0")

In [10]:
m.model_instances

{'first': <DB2Gams_l3.gams_model at 0x28bdf25d248>}

#### Sneaky solve to adjust sigmas into desired values

The model could be solved using elasticities of 0.0001. Let's check if it can be solved when these are set to higher numbers, implying a solution farther from the starting values (which, as mentioned, were set based on Leontief).

In [11]:
# condition = m.model.settings.databases["Abatement"]["sigma"].vals.index.str.startswith("C_E") & ~m.model.settings.databases["Abatement"]["sigma"].vals.index.str.contains("base")

In [12]:
#Sigmas in MNL nests of end-of-pipe cannot be Leontief, because share-parameters are inconsistent with quantities, and so we need prices to compensate.


In [13]:
# for sigma_val in range(0, 450, 10):
#     sigma_val = sigma_val/100
#     if sigma_val == 0:
#         sigma_val = 0.0001
# #     for MNL_sigma in [0.1, 1]:

#     #Set values
#     #m.model.settings.databases["Abatement"]["sigma"].vals.loc[condition] = MNL_sigma
#     #m.model.settings.databases["Abatement"]["sigma"].vals.loc[~condition] = sigma_val
# #     m.get("sigma")[[t for t in m.get("sigma").index if t not in m.get("EOP_C_subset")]] = sigma_val
#     m.model.settings.databases["ID_0"]["sigma"].vals.loc[:] = sigma_val
#     m.model.settings.databases["ID_0"]["eta"].vals.loc[:] = -sigma_val
# #     m.get("sigma")[m.get("EOP_C_subset")] = sigma_val
# #     m.get("eta")[m.get("eta").index.str.startswith("EOP")] = -sigma_val

#     #Try to solve
#     m.write_and_run()

#     #How did it go?
#     if m.model_instances["baseline"].modelstat == 16.0:
#         print("\nSUCCESS:")
#         print("Sigma/eta:", round(sigma_val, 4))
#         #print("Sigma/eta (MNL):", round(MNL_val, 4))
#     else:
#         print("\nFAIL:")
#         print("Sigma/eta:", round(sigma_val, 4))
#         #print("Sigma/eta: (MNL)", round(MNL_sigma, 4))

As can be seen, the model cannot be solved when the elasticities get too large. To accomodate this, we replace each iteration's starting values with the previous iteration's solution, while gradually increasing $\sigma$ and $\mu$.
This allows us to set these to (hopefully) any value. This "gradual changing of parameters" is called 'sneaky solve'. :)

### Sneaky solve: Gradual increase in sigma

In [14]:
db_star = DataBase.GPM_database(**{'name': 'somerandomname'})

In [15]:
sigma_star = m.get("sigma").copy()
eta_star = m.get("eta").copy()
mu_star = m.get("mu").copy()
# sigmaG_star = m.get("sigmaG").copy()

In [16]:
#Energy services combine their underlying components with mild substitution possibilities
# sigma_star[m.get("kno_ID_EC")] = 0.1

In [17]:
#All CET splits get more substitution
eta_star[:] = -2

In [18]:
#The IO technology treats its outputs as perfect substitutes. This ensures they have the same price in equiliibrum.
#eta_star[m.get("kno_no_ID_BX")] = -10

In [19]:
#Components have a large degree of substitution between overlapping technology goods:
sigma_star[m.get("kno_ID_CU")] = 0.9 #Virker ikke når denne sættes til 1.3

In [20]:
sigma_star[m.get("kno_no_ID_BX")] = 0.5
sigma_star["Y"] = 0.2

In [21]:
#gammas should be 1 from the beginning, so that baseline U has same prices as their baseline technology. 
#the share parameter of all baseline technology goods U in the C->U nests should be 1.
#U0->C
mu_star[mu_star.index.get_level_values(0).isin(m.get("bra_no_ID_BU")) & mu_star.index.get_level_values(1).isin(m.get("kno_no_ID_CU"))] = 1.0
#gammas, B->U0
mu_star[m.get("map_ID_BU")] = 1.0

In [22]:
#Make adoption of end-of-pipe techs closer to discrete:
# sigmaG_star = m.get("sigmaG")

In [23]:
db_star["sigma"] = sigma_star
db_star["eta"] = eta_star
# db_star["mu"] = mu_star

Run again, while adding a checkpoint that we can use afterwards

In [24]:
name='v0'
m.write_and_run(name=name, add_checkpoint=name)
m.checkpoints['v1'] = m.model_instances[name].ws.add_checkpoint()

In [25]:
m.model_instances[name].solve_sneakily(db_star=db_star,from_cp = True, cp_init = m.checkpoints[name], options_run={'checkpoint': m.checkpoints['v1']})

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [27]:
name = "first"
#Fejler med noget "delimiter" fejl
#m.write_and_run(name="test", add_checkpoint="test", options_add={"checkpoint":m.checkpoints["leontief"]})
m.write_and_run(name=name, add_checkpoint="leontief")

In [27]:
m.checkpoints[name] = m.model_instances[name].ws.add_checkpoint()

Solve sneakily, gradually increasing sigmas towards their desired values (which themselves are stated in `db_star`)

In [28]:
m.model_instances[name].solve_sneakily(db_star=db_star, options_run={"checkpoint":m.checkpoints[name]}, kwargs_shock={"n_steps":99})

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [29]:
mi = m.model_instances[name]
# newjob = mi.ws.add_job_from_string("sets a,b;",**{'checkpoint': m.checkpoints[name]})
# newjob.run()

Check if the model ran succesfully:

In [32]:
modelstat = [t for t in m.model_instances[name].out_db.symbols.keys() if "modelstat" in t][0]
solvestat = [t for t in m.model_instances[name].out_db.symbols.keys() if "solvestat" in t][0]

In [33]:
if m.model_instances[name].out_db.series[modelstat].vals == 16:
    print("Ran successfully with modelstat " + str(m.model_instances[name].out_db.series[modelstat].vals))
    print("Solvestat was " + str(m.model_instances[name].out_db.series[solvestat].vals))

Ran successfully with modelstat 16.0
Solvestat was 1.0


In [34]:
m.model_instances[name].out_db.get("currapp_ID")

n
ID_t2_EL    0.009977
Name: currapp_ID, dtype: float64

The changes in e.g. demanded quantities, from the initial point (the solution where sigmas were ~0) are:

In [36]:
m.model_instances[name].out_db["qD"].vals - m.get("qD")

n
C0_EL                        8.818376
C_EL_1                       1.605442
C_EL_2                       5.613647
EL                          16.037464
EOP_t2                     -99.999996
EOP_t2_K                  -549.999979
EOP_t2_inp3                -99.999996
ID_t1                       -0.999766
ID_t1_K                     -8.997897
ID_t1_electricity           -0.999766
ID_t2                       -1.000000
ID_t2_K                    -24.000000
ID_t2_electricity           -1.000000
U0_ID_C0_EL                  8.818376
U0_ID_C_EL_1                 2.605442
U0_ID_C_EL_2                 6.613413
U_EOP_t2_1                 -99.999996
U_ID_t1_1                   -0.999766
U_ID_t2_1                   -1.000000
Y                            0.000000
Y_in_K                       6.490362
Y_in_electricity             6.490362
Y_in_inp3                    6.490362
Y_in_oil                   -17.083457
basetech_EL                -47.027391
basetech_EL_K               -8.987527
basetech_E

while demand itself ended in the following quantities:

In [38]:
m.model_instances[name].out_db["qD"].vals

n
EOP_t2                     3.840366e-06
basetech_EL_oil            8.298665e+00
C_EL_1                     1.160544e+01
basetech_EL_electricity    1.551247e+01
EOP_t2_inp3                3.841085e-06
C_EL_2                     4.061365e+01
U_ID_t1_1                  2.336630e-04
U0_ID_C_EL_1               1.160544e+01
ID_t1                      2.336630e-04
basetech_EL_inp3           1.551247e+01
ID_t1_electricity          2.337169e-04
U_ID_t2_1                  2.448160e-10
basetech_EL                5.097261e+01
C0_EL                      6.381838e+01
Y_in_electricity           1.064904e+02
ID_t2_electricity          2.448948e-10
Y                          5.000000e+02
EL                         1.160375e+02
ID_t2                      2.448160e-10
ID_t2_K                    5.877475e-09
ID_t1_K                    2.103452e-03
basetech_EL_K              1.551247e+01
EOP_t2_K                   2.112596e-05
Y_in_K                     1.064904e+02
Y_in_oil                   8.291654e+0

And let's confirm that our sigmas actually ended in the desired spots:

In [39]:
m.model_instances[name].out_db["sigma"].vals

n
EOP_t2         0.0001
C_EL_1         0.9000
C_EL_2         0.9000
ID_t1          0.0001
basetech_EL    0.5000
C0_EL          0.9000
C_CO2_1        2.0000
Y              0.2000
EL             0.0001
ID_t2          0.0001
Name: sigma, dtype: float64

In [40]:
#Save the final values from the sneaky loop
db = m.model_instances['sneaky'].out_db
m.model.database.merge_dbs(m.model.database,db,'second')

In [28]:
m.reset_settings()
# m.setstate("EOP")
# m.write_and_run(name="with_substitution", overwrite=True)

In [29]:
#midlertidigt stop her. Behøver ikke kalibrering for at lave figurer. 
m.export()

'C:\\Users\\zgr679\\Documents\\GitHub\\GPM_v05\\examples\\Abatement\\Data\\..\\gamsmodels\\Main\\gmspython_Abatement'

In [43]:
# m.get("currapp_ID_modified")

# m.model_instances["with_substitution"].out_db.get("currapp_ID_modified")

# inputfile = False
# if inputfile == "techdata_new2ID_only_simple_onewith2E.xlsx":
#     numerator = m.get("mu")["U_ID_t3_2", "C_EL_4"] * math.exp((m.get("PwThat")["C_EL_4"] - m.get("PwThat")["U_ID_t3_2"])*m.get("sigma")["C_EL_4"])
#     denominator_1 = m.get("mu")["U_ID_t3_2", "C_EL_4"] * math.exp((m.get("PwThat")["C_EL_4"] - m.get("PwThat")["U_ID_t3_2"])*m.get("sigma")["C_EL_4"])
#     denominator_2 = m.get("mu")["U0_ID_C_EL_4", "C_EL_4"] * math.exp((m.get("PwThat")["C_EL_4"] - 
#                                                                   (m.get("gamma_tau")["ID_t3"] * m.get("PwThat")["basetech_EL"]))*m.get("sigma")["C_EL_4"])
#     CoverE = m.get("qD")["C_EL_4"]/m.get("qD")["EL"]
#     number = (numerator / (denominator_1 + denominator_2) * CoverE)
#     if round(number, 8) == round(m.get("currapp_ID_modified")["ID_t3_EL"], 8):
#         print("Yes")
#     else:
#         print(number)
#         print(m.get("currapp_ID_modified")["ID_t3_EL"])

In [44]:
#Replace current applications (modified) with the ones we ended with for now:
db = DataBase.GPM_database()
db["currapp_ID_modified"] = m.get("currapp_ID").copy()
m.model.database.merge_dbs(m.model.database, db, 'second')

In [45]:
m.reset_settings()

In [46]:
m.setstate("EOPcalibrate")

In [47]:
# m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
# m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001
#m.model.settings.databases["Abatement"]["sigma"].vals[condition] = 0.2
#m.write_and_run(options_run={'output':sys.stdout})
m.write_and_run(name="calib", overwrite=True)
if m.model_instances["calib"].modelstat == 2.0:
    print("\nNPL problem solved successfully! The modelstat was 2.0")
else:
    print("Failed, loser")


NPL problem solved successfully! The modelstat was 2.0


In [48]:
m.export()

'C:\\Users\\zgr679\\Documents\\GitHub\\GPM_v05\\examples\\Abatement\\Data\\..\\gamsmodels\\Main\\gmspython_Abatement'